In [1]:
import os

os.environ["MLFLOW_TRACKING_URL"]="https://dagshub.com/cmatias/datascienceproject.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="cmatias"
os.environ["MLFLOW_TRACKING_PASSWORD"]="b570118b3787b1909d48aa33c793948cfa6dc206"

In [2]:
%pwd

'c:\\Users\\cmatias\\Desktop\\MLOPS\\datascienceproject\\research'

In [3]:
os.chdir("../")
%pwd

'c:\\Users\\cmatias\\Desktop\\MLOPS\\datascienceproject'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str
    

In [5]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories, save_json

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/cmatias/datascienceproject.mlflow"
        )
        return model_evaluation_config

In [7]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib


In [8]:
class ModelEvalution:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        X_test = test_data.drop(self.config.target_column, axis=1)
        y_test = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities = model.predict(X_test)
            (rmse, mae, r2) = self.eval_metrics(y_test, predicted_qualities)

            scores = {
                "rmse": rmse,
                "r2": r2,
                "mae": mae
            }
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)


            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)

            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticNetModel")
            else:
                mlflow.sklearn.log_model(model, "model")
            

In [9]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvalution(model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

2025-03-01 20:04:57,333: INFO: common: Reading yaml file from config\config.yaml
2025-03-01 20:04:57,340: INFO: common: Reading yaml file from params.yaml
2025-03-01 20:04:57,345: INFO: common: Reading yaml file from schema.yaml
2025-03-01 20:04:57,348: INFO: common: Created directory: artifacts
2025-03-01 20:04:57,352: INFO: common: Created directory: artifacts/model_evaluation


2025-03-01 20:04:58,208: INFO: common: Json file saved to artifacts\model_evaluation\metrics.json


2025/03/01 20:05:11 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
